In [1]:
import psycopg2
import psycopg2.extras
import pandas 
import numpy as np

import pandas as pd
import os


import requests
import astropy.io.fits as fits
import glob 

In [2]:
db = psycopg2.connect("dbname='decat' user='decat_ro' password='oe0f7cc0uaa3' host='decatdb.lbl.gov'")
pandas.set_option('display.max_rows', 1060)

FIELDS:

COSMOS-1    150.0   3.1

COSMOS-2    149.22  1.75

COSMOS-3    150.78  1.75

DECaPS-East 270.89  -29.53

ELAIS-E1    7.8746  -43.0096

ELAIS-E2    9.5     -43.9980

DECaPS-West 116.32  -26.25

In [3]:
#Divides up files that were pulled by filter

def dividebyfilter(surveyname): 
    green = surveyname[(surveyname['filter'] == "g")] 
    red = surveyname[(surveyname['filter'] == "r")] 
    i = surveyname[(surveyname['filter']== "i")] 
    #z = surveyname[(surveyname['filter'] == "z")] 
    return green, red, i 

In [4]:
#The trimmed mean is computed by excluding the specified percentage of the largest and smallest values 
#from the sample and taking the mean of the remaining values

def trimmean(arr, percent):
    n = len(arr)
    k = int(round(n*(float(percent)/100)/2))
    return np.mean(arr[k+1:n-k])

In [5]:
#Determines the cutoff values for images using trimmed mean

def magzpvalues(green_arr, red_arr, i_arr, greensky_arr, redsky_arr, isky_arr):
    greenavg = round(trimmean(green_arr, 95),1) 
    greenhigh = round((greenavg) +.1,1)
    greenlow = round((greenavg - .1),1)
    greenskysig = round(trimmean(greensky_arr, 10))
    
    
    redavg = round(trimmean(red_arr, 95),1) 
    redhigh = round(redavg +.1,1)
    redlow = round((redavg - .1),1)
    redskysig = round(trimmean(redsky_arr, 10))
    
    i_avg = round(trimmean(i_arr, 95),1)
    ihigh = round(i_avg +.1,1)
    ilow = round((i_avg - .1),1)
    iskysig = round(trimmean(isky_arr, 10))
    
    
    print('green_avg: %s' %greenavg)
    print('green_lowzp: %s' %greenlow)
    print('green_highmagzp: %s' %greenhigh)
    print('greenskysig: %s' %greenskysig)
    print('----')
    print('red_avg: %s' %redavg)
    print('red_lowzp: %s' %redlow)
    print('red_highmagzp: %s' %redhigh)
    print('redskysig: %s' %redskysig)
    print('----')
    print('i_avg: %s' %i_avg)
    print('i_lowzp: %s' %ilow)    
    print('i_highmagzp: %s' %ihigh)
    print('iskysig: %s' %iskysig)
    print('----')
    

    
    return greenhigh, greenlow, greenskysig, redhigh, redlow, redskysig, ihigh, ilow, iskysig



In [9]:
#Creates 3 csv files in g, r, and i filters in the folder of the field with the cutoff values. This gives the good images for stacking 
def splitcsv(field_df, fieldname, ccdnum, g_lowzp, g_highzp, g_skysig, r_lowzp, r_highzp, r_skysig, i_lowzp, i_highzp, i_skysig):
    green = field_df[(field_df['filter'] == "g")] 
    red = field_df[(field_df['filter'] == "r")] 
    i = field_df[(field_df['filter']== "i")] 
    
    
    #take out all the "bad" data for each filter
    green = green[(green['magzp']<g_highzp) & (green['magzp']>g_lowzp)]
    green = green[(green['seeing']<1.61)]
    green = green[(green['skysig']<g_skysig)]
    
    
    red = red[(red['magzp']<r_highzp) & (red['magzp']>r_lowzp)]
    red = red[(red['seeing']<1.61)]
    red = red[(red['skysig']<r_skysig)]
    
    i = i[(i['magzp']<i_highzp) & (i['magzp']>i_lowzp)]
    i = i[(i['seeing']<1.61)]
    i = i[(i['skysig']<i_skysig)]
    
    
    #save csv files to fieldname folder 
    greenpath = os.path.join('/global/u2/a/autumnaw/WorkinginDecatDatabase/DECATfields/', fieldname, fieldname + '_' + str(ccdnum) + '_g.csv' )
    redpath = os.path.join('/global/u2/a/autumnaw/WorkinginDecatDatabase/DECATfields/', fieldname, fieldname + '_' + str(ccdnum) + '_r.csv' )
    ipath = os.path.join('/global/u2/a/autumnaw/WorkinginDecatDatabase/DECATfields/', fieldname, fieldname + '_' + str(ccdnum) + '_i.csv' )
    
    
    green.to_csv(greenpath)
    red.to_csv(redpath)
    i.to_csv(ipath)
    
    return green, red, i

In [14]:
#31 and 61 are not chipnums. you can also change what chipnums you want to pull
chipnums = ['01', '02', '03', '04', '05', '06', '07', '08', '09', 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 62]

In [15]:
for ccdnum in chipnums:   
    
    #pulls all the files from every survey for all the chipnums from 1 to 62
    
    db.rollback()
    cursor = db.cursor( cursor_factory=psycopg2.extras.RealDictCursor )
    #COSMOS1 150.0 3.1
    cursor.execute( "SELECT e.filename,e.ra,e.dec, e.filter, e.proposalid, "
                    "   i.basename, i.seeing, i.skysig, i.magzp "
                    "FROM exposures e "
                    "INNER JOIN images i ON i.exposure_id=e.id "
                    "WHERE e.ra>149.5 AND e.ra<150.5 AND e.dec>2.6 AND e.dec<3.6 "
                    "       AND e.proposalid IN ('2021B-0149','2022A-724693') "
                    "       AND i.ccdnum='%s'        "
                    "ORDER BY e.filter,e.mjd " %ccdnum)
    COSMOS1 = pandas.DataFrame( cursor.fetchall() )
    COSMOS1.name = 'COSMOS1'
    #COSMOS1.to_csv('/global/u2/a/autumnaw/WorkinginDecatDatabase/DECATfields/COSMOS1/COSMOS1_'+ str(ccdnum) + '.csv')


    #COSMOS2 149.22 1.75
    cursor.execute( "SELECT e.filename,e.ra,e.dec, e.filter, e.proposalid, "
                    "   i.basename, i.seeing, i.skysig, i.magzp "
                    "FROM exposures e "
                    "INNER JOIN images i ON i.exposure_id=e.id "
                    "WHERE e.ra>148 AND e.ra<149.5 AND e.dec>1.70 AND e.dec<1.80 "
                    "       AND e.proposalid IN ('2021B-0149','2022A-724693') "
                    "       AND i.ccdnum='%s'         "
                    "ORDER BY e.filter,e.mjd " %ccdnum)
    COSMOS2 = pandas.DataFrame( cursor.fetchall() )
    COSMOS2.name = 'COSMOS2'
    #COSMOS2.to_csv('/global/u2/a/autumnaw/WorkinginDecatDatabase/DECATfields/COSMOS2/COSMOS2_'+ str(ccdnum) + '.csv')



    #COSMOS3 150.78 1.75
    cursor.execute( "SELECT e.filename,e.ra,e.dec, e.filter, e.proposalid, "
                    "   i.basename, i.seeing, i.skysig, i.magzp "
                    "FROM exposures e "
                    "INNER JOIN images i ON i.exposure_id=e.id "
                    "WHERE e.ra>150 AND e.ra<151 AND e.dec>1.70 AND e.dec<1.80 "
                    "       AND e.proposalid IN ('2021B-0149','2022A-724693') "
                    "       AND i.ccdnum='%s'        "
                    "ORDER BY e.filter,e.mjd "  %ccdnum)
    COSMOS3 = pandas.DataFrame( cursor.fetchall() )
    COSMOS3.name = 'COSMOS3'
    #COSMOS3.to_csv('/global/u2/a/autumnaw/WorkinginDecatDatabase/DECATfields/COSMOS3/COSMOS3_'+ str(ccdnum) + '.csv')


    #DECAPS-EAST 270.89  -29.53
    cursor.execute( "SELECT e.filename,e.ra,e.dec, e.filter, e.proposalid, "
                    "   i.basename, i.seeing, i.skysig, i.magzp "
                    "FROM exposures e "
                    "INNER JOIN images i ON i.exposure_id=e.id "
                    "WHERE e.ra>270 AND e.ra<271 AND e.dec>-30 AND e.dec<-28 "
                    "       AND e.proposalid IN ('2021B-0149','2022A-724693') "
                    "       AND i.ccdnum= '%s'       " 
                    "ORDER BY e.filter,e.mjd " %ccdnum)

    DECAPS_EAST = pandas.DataFrame( cursor.fetchall() )
    DECAPS_EAST.name = 'DECAPS_EAST'
    #DECAPS_EAST.to_csv('/global/u2/a/autumnaw/WorkinginDecatDatabase/DECATfields/DECAPS_EAST/DECAPS_EAST_'+ str(ccdnum) + '.csv')

    #ELAIS-E1    7.8746  -43.0096

    db.rollback()
    cursor = db.cursor( cursor_factory=psycopg2.extras.RealDictCursor )
    #cursor.execute("SELECT * FROM image")
    cursor.execute( "SELECT e.filename,e.ra,e.dec, e.filter, e.proposalid, "
                    "   i.basename, i.seeing, i.skysig, i.magzp "
                    "FROM exposures e "
                    "INNER JOIN images i ON i.exposure_id=e.id "
                    "WHERE e.ra>7 AND e.ra<8 AND e.dec>-43.1 AND e.dec<-42 "
                    "       AND e.proposalid IN ('2021B-0149','2022A-724693') "
                    "       AND i.ccdnum='%s'         "
                    "ORDER BY e.filter,e.mjd " %ccdnum )


    ELAIS_E1  = pandas.DataFrame( cursor.fetchall() )
    ELAIS_E1.name = 'ELAIS_E1'
    #ELAIS_E1 .to_csv('/global/u2/a/autumnaw/WorkinginDecatDatabase/DECATfields/ELAIS_E1/ELAIS_E1_'+ str(ccdnum) + '.csv')

    #ELAIS-E2 9.5 -43.9980

    db.rollback()
    cursor = db.cursor( cursor_factory=psycopg2.extras.RealDictCursor )
    #cursor.execute("SELECT * FROM image")
    cursor.execute( "SELECT e.filename,e.ra,e.dec, e.filter, e.proposalid, "
                    "   i.basename, i.seeing, i.skysig, i.magzp "
                    "FROM exposures e "
                    "INNER JOIN images i ON i.exposure_id=e.id "
                    "WHERE e.ra>9 AND e.ra<10 AND e.dec>-44.0 AND e.dec<-43 "
                    "       AND e.proposalid IN ('2021B-0149','2022A-724693') "
                    "       AND i.ccdnum='%s'         "
                    "ORDER BY e.filter,e.mjd " %ccdnum )
    ELAIS_E2  = pandas.DataFrame( cursor.fetchall() )
    ELAIS_E2.name = 'ELAIS_E2'
    #ELAIS_E2.to_csv('/global/u2/a/autumnaw/WorkinginDecatDatabase/DECATfields/ELAIS_E2/ELAIS_E2_'+ str(ccdnum) + '.csv')


    #DECAPS-WEST 116.32  -26.25
    cursor.execute( "SELECT e.filename,e.ra,e.dec, e.filter, e.proposalid, "
                    "   i.basename, i.seeing, i.skysig, i.magzp "
                    "FROM exposures e "
                    "INNER JOIN images i ON i.exposure_id=e.id "
                    "WHERE e.ra>116 AND e.ra<118 AND e.dec>-27 AND e.dec<-25 "
                    "       AND e.proposalid IN ('2021B-0149','2022A-724693') "
                    "       AND i.ccdnum='%s'         "
                    "ORDER BY e.filter,e.mjd " %ccdnum )
    DECAPS_WEST = pandas.DataFrame( cursor.fetchall() )
    DECAPS_WEST.name = 'DECAPS_WEST'
    #DECAPS_WEST.to_csv('/global/u2/a/autumnaw/WorkinginDecatDatabase/DECATfields/DECAPS_WEST/DECAPS_WEST_'+ str(ccdnum) + '.csv')

    
    
    
    field_dataframes = [COSMOS1, COSMOS2, COSMOS3, DECAPS_EAST, ELAIS_E1, ELAIS_E2, DECAPS_WEST] 
    
    
    
    
    #loops through all the dataframes for the specified chipnum using defined functions above
    for k in field_dataframes: 
        green, red, i = dividebyfilter(k) 
        fieldname = k.name

        print(fieldname)
        print('Chipnum ' + str(ccdnum))
        greenhigh, greenlow, greenskysig, redhigh, redlow, redskysig, ihigh, ilow, iskysig = magzpvalues(green['magzp'], red['magzp'], i['magzp'], green['skysig'], red['skysig'], i['skysig'])

        filepath = os.path.join('/global/u2/a/autumnaw/WorkinginDecatDatabase/DECATfields/', fieldname, fieldname + '_' + str(ccdnum) + '.csv')


        g, r, i = splitcsv(k, fieldname, ccdnum, greenlow, greenhigh, greenskysig, redlow, redhigh, redskysig, ilow, ihigh, iskysig)
        print( 'Saved ' + fieldname + ' to filepath ' + filepath)
        print('--------------')
        
        
    

COSMOS1
Chipnum 01
green_avg: 29.0
green_lowzp: 28.9
green_highmagzp: 29.1
greenskysig: 12
----
red_avg: 29.5
red_lowzp: 29.4
red_highmagzp: 29.6
redskysig: 19
----
i_avg: 29.9
i_lowzp: 29.8
i_highmagzp: 30.0
iskysig: 28
----
Saved COSMOS1 to filepath /global/u2/a/autumnaw/WorkinginDecatDatabase/DECATfields/COSMOS1/COSMOS1_01.csv
--------------
COSMOS2
Chipnum 01
green_avg: 29.0
green_lowzp: 28.9
green_highmagzp: 29.1
greenskysig: 12
----
red_avg: 29.5
red_lowzp: 29.4
red_highmagzp: 29.6
redskysig: 18
----
i_avg: 29.9
i_lowzp: 29.8
i_highmagzp: 30.0
iskysig: 29
----
Saved COSMOS2 to filepath /global/u2/a/autumnaw/WorkinginDecatDatabase/DECATfields/COSMOS2/COSMOS2_01.csv
--------------
COSMOS3
Chipnum 01
green_avg: 29.0
green_lowzp: 28.9
green_highmagzp: 29.1
greenskysig: 12
----
red_avg: 29.5
red_lowzp: 29.4
red_highmagzp: 29.6
redskysig: 19
----
i_avg: 29.9
i_lowzp: 29.8
i_highmagzp: 30.0
iskysig: 29
----
Saved COSMOS3 to filepath /global/u2/a/autumnaw/WorkinginDecatDatabase/DECATfiel